# Lab 1: Introducción a Stable Diffusion

Referencias:

- [Stable Diffusion Videos 📽 / nateraw - GitHub ](https://github.com/nateraw/stable-diffusion-videos)
- [A walk through latent space with Stable Diffusion - Keras](https://keras.io/examples/generative/random_walks_with_stable_diffusion/)

## Parte 3: Generación de Videos con Stable Diffusion 📽

In [ ]:
!pip install -q -U transformers accelerate ftfy fpuna-stable-diffusion diffusers==0.14.0 av

Los modelos de generación de imágenes aprenden una representación "latente" del mundo visual: un vector de baja dimensión donde cada punto se puede convertir en una imagen. Ir desde ese punto latente a una imagen se llama "decodificación". En Stable Diffusion 🧨, esto es manejado por el "decodificador".

Esta variedad latente de imágenes es continua e interpolativa, significando que:

1. Moverse un poquito en el espacio latente hace que la imagen correspondiente se cambie un poquito (continuidad)
2. Para cualquier dos puntos **A** y **B** (por ejemplo, cualquier dos imágenes), es posible moverse de **A** a **B** por un camino donde cada punto intermedio esta también en el espacio latente (por ejemplo, también otra imagen valida). Los puntos intermedios pueden ser llamados "interpolaciones" entre las dos imágenes de inicio. 

Stable Diffusion 🧨 no solo es un modelo de imágenes, sino también, un modelo de lenguaje natural. Posee dos espacios latentes: el espacio de representación de imagen aprendida por el codificador durante el entrenamiento, y el espacio latente del texto que es aprendida usando la combinación de pre-entrenamiento y ajuste duante el entrenamiento.

_Latent space walking_, o, _exploración en el espacio latente_, es el proceso de obtener un punto de muestra en el espacio latente e ir cambiando la representación latente de forma incrementada. Su aplicación más común es generar animaciones donde cada punto muestreado es alimentado al decodificador y es guardado como un fotograma en la animación final.

Para representaciones latentes de alta calidad, esto produce animaciones que parecen coherentes. Estas animaciones pueden proveer una cierta intuición del espacio latente, y que puede ultimamente llevar a mejoras en el proceso de entrenamiento.


<p style="text-align:center;"><img src="https://keras.io/img/examples/generative/random_walks_with_stable_diffusion/panda2plane.gif" alt="panda a avión"  width="300" /></p>

### Creamos nuestro modelo de Stable Diffusion 🧨

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

### Interpolando entre los text prompts 

En Stable Diffusion, un text prompt es primero codificado en un vector, y esa codificación es luego utilizada para guiar el proceso de difusión. El vector latente tiene un tamaño de 77x768 (es una matriz!), y cuando damos a Stable Diffusion un text prompt, estamos generando imágenes desde solamente un punto en el espacio latente.

Para explorar más este espacio latente, podemos interpolar entre dos codificaciones de texto y generar imágenes en esos puntos interpolados.

In [ ]:
import numpy as np

interpolation_steps = 5
prompt_1 = "A watercolor painting of a Golden Retriever at the beach"
prompt_2 = "A still life DSLR photo of a bowl of fruit"

tokenized_1 = pipe.tokenizer(prompt_1, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")
tokenized_2 = pipe.tokenizer(prompt_2, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")

encoding_1 = pipe.text_encoder(tokenized_1.input_ids.to("cuda"))[0][0].cpu().detach().numpy()
encoding_2 = pipe.text_encoder(tokenized_2.input_ids.to("cuda"))[0][0].cpu().detach().numpy()

interpolated_encodings = np.linspace(encoding_1, encoding_2, interpolation_steps)

In [ ]:
interpolated_encodings.shape

Una vez que hemos interpolado las codificaciones latentes, podemos generar imágenes desde cualquier punto. Nota que para mantener cierta estabilidad entre las imágenes resultantes, debemos mantener el ruido de difusión constante entre las imágenes.

In [ ]:
generator = torch.Generator().manual_seed(12345)
noise = torch.randn((1, 4, 64, 64), generator=generator, dtype=torch.float16)
noise = torch.cat([noise]*interpolation_steps, dim=0)

images = pipe(prompt_embeds=torch.from_numpy(interpolated_encodings).to("cuda"),
              latents=noise
             ).images

Ahora que generamos algunas imágenes interpoladas, echemos un vistazo!

Durante el laboratorio, vamos a estar exportando las secuencias de imagenes como gifs para que puedan verse facilmente con cierto contexto temporal. Para las secuencias de imágenes donde la primera y la ultima imagen no coincide conceptualmente, vamos hacer que se repita el video.

In [ ]:
from fpuna_stable_diffusion.utils import image_grid

image_grid(images, rows=1, cols=interpolation_steps)

In [ ]:
def export_as_gif(filename, images, frames_per_second=10, rubber_band=False):
    if rubber_band:
        images += images[2:-1][::-1]
    images[0].save(
        filename,
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )
    
export_as_gif(
    "doggo-and-fruit-5.gif",
    images,
    frames_per_second=2,
    rubber_band=True,
)

In [ ]:
from IPython.display import Image as IImage
IImage("doggo-and-fruit-5.gif")

Los resultados pueden parecer sorprendente. Generalmente, interpolar entre prompts produce imágenes coherentes, y demuestran muy de seguido un cambio de concepto progresivo entre los contenidos de los dos textos. Esto es indicativo de un espacio de representación de alta calidad, que refleja cercanamente la estructura natural del mundo visual.

Para visualizar mejor, tenemos que hacer una interpolación mucho más fina, utilizando cientos de pasos de interpolación. Para poder mantener el tamaño de las interpolaciones baja (para evitar que la GPU se quede corto de memoria), se requiere lotear manualmente las interpolaciones.

In [ ]:
interpolation_steps = 30
batch_size = 3
batches = interpolation_steps // batch_size

# Generamos de nuevo el ruido
generator = torch.Generator().manual_seed(12345)
noise = torch.randn((1, 4, 64, 64), generator=generator, dtype=torch.float16)

# Generamos las interpolaciones y la dividimos en lotes
interpolated_encodings = torch.from_numpy(np.linspace(encoding_1, encoding_2, interpolation_steps))
interpolated_encodings = interpolated_encodings.to("cuda") # Hay que llevar a GPU
batched_encodings = interpolated_encodings.chunk(batches)

# Generamos para cada lote
images = []
for batch in batched_encodings:
    noise_batch = torch.cat([noise] * batch.shape[0], dim=0)
    images += pipe(prompt_embeds=batch,
                   latents=noise_batch,
                   num_inference_steps=25).images
    
export_as_gif("doggo-and-fruit-30.gif", images, rubber_band=True)

In [ ]:
IImage("doggo-and-fruit-30.gif")

El gif resultante muestra un cambio más claro y coherente entre los dos textos (ten en cuenta que bajamos el numero de pasos). Prueba tus propios textos y experimenta !

Incluso podemos extender este concepto para más de una imagen. Por ejemplo, podemos interpolar entre cuatro textos:

In [ ]:
prompt_1 = "A watercolor painting of a Golden Retriever at the beach"
prompt_2 = "A still life DSLR photo of a bowl of fruit"
prompt_3 = "The eiffel tower in the style of starry night"
prompt_4 = "An architectural sketch of a skyscraper"

interpolation_steps = 6
batch_size = 4
batches = (interpolation_steps**2) // batch_size

Codificamos los textos.

In [ ]:
tokenized_1 = pipe.tokenizer(prompt_1, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")
tokenized_2 = pipe.tokenizer(prompt_2, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")
tokenized_3 = pipe.tokenizer(prompt_3, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")
tokenized_4 = pipe.tokenizer(prompt_4, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")

encoding_1 = pipe.text_encoder(tokenized_1.input_ids.to("cuda"))[0][0].cpu().detach().numpy()
encoding_2 = pipe.text_encoder(tokenized_2.input_ids.to("cuda"))[0][0].cpu().detach().numpy()
encoding_3 = pipe.text_encoder(tokenized_3.input_ids.to("cuda"))[0][0].cpu().detach().numpy()
encoding_4 = pipe.text_encoder(tokenized_4.input_ids.to("cuda"))[0][0].cpu().detach().numpy()

Interpolamos los textos.

In [ ]:
interpolated_encodings = torch.from_numpy(np.linspace(
    np.linspace(encoding_1, encoding_2, interpolation_steps),
    np.linspace(encoding_3, encoding_4, interpolation_steps),
    interpolation_steps
))

interpolated_encodings = interpolated_encodings.reshape(
    (interpolation_steps**2, 77, 768)
).to("cuda")

Dividimos en lotes las interpolaciones.

In [ ]:
batched_encodings = interpolated_encodings.chunk(batches)

Procedemos a generar las imágenes interpoladas, lote por lote.

In [ ]:
images = []
for batch in batched_encodings:
    noise_batch = torch.cat([noise] * batch.shape[0], dim=0)
    images += pipe(prompt_embeds=batch,
                   latents=noise_batch,
                   num_inference_steps=50).images

Veamos las imágenes que generamos.

In [ ]:
from fpuna_stable_diffusion.utils import image_grid

grid = image_grid(images, rows=interpolation_steps, cols=interpolation_steps).resize((800, 800))
grid.save("4-way-interpolation.jpg")

grid

Podemos también interpolar mientras permitimos que el ruido de difusión varie si es que dejamos de pasar el ruido inicial.

In [ ]:
images = []
for batch in batched_encodings:
    noise_batch = torch.cat([noise] * batch.shape[0], dim=0)
    images += pipe(prompt_embeds=batch,
                   # latents=noise_batch,
                   num_inference_steps=50).images
    
image_grid(images, rows=interpolation_steps, cols=interpolation_steps).resize((600, 600))

### Una caminata alrededor de un texto

Nuestro siguiente experimento será ir de caminata alrededor del espacio latente empezando por un punto producido por un texto particular.

In [ ]:
walk_steps = 30
batch_size = 4
batches = walk_steps // batch_size
step_size = 0.005

In [ ]:
tokenized = pipe.tokenizer("The Eiffel Tower in the style of starry night", padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")
encoding = pipe.text_encoder(tokenized.input_ids.to("cuda"))[0][0]

delta = torch.ones_like(encoding) * step_size

In [ ]:
walked_encodings = []

for step_index in range(walk_steps):
    walked_encodings.append(encoding + delta * step_index)
    
walked_encodings = torch.stack(walked_encodings)
batched_encodings = walked_encodings.chunk(batches)

In [ ]:
# Generamos de nuevo el ruido
generator = torch.Generator().manual_seed(42)
noise = torch.randn((1, 4, 64, 64), generator=generator, dtype=torch.float16)

# Apagamos el filtro para no tener problemas
def dummy(images, **kwargs):
    return images, False

pipe.safety_checker = dummy

images = []
for batch in batched_encodings:
    noise_batch = torch.cat([noise] * batch.shape[0], dim=0)
    images += pipe(prompt_embeds=batch,
                   latents=noise_batch,
                   num_inference_steps=30).images

In [ ]:
export_as_gif("eiffel-tower-starry-night.gif", images, rubber_band=True)

In [ ]:
IImage("eiffel-tower-starry-night.gif")

### Caminata circular sobre el espacio del ruido de difusión para un mismo texto

Nuestro experimento final será pegarse a un solo texto y explorar la variedad de imágenes que el modelo de difusión puede producir para ese texto. Podemos hacer esto controlando el ruido que es usado para inicial el proceso de difusión.

In [ ]:
import math

prompt = "An oil paintings of cows in a field next to a windmill in Holland"

tokenized = pipe.tokenizer(prompt, padding="max_length", max_length=pipe.tokenizer.model_max_length, truncation=True, return_tensors="pt")
encoding = pipe.text_encoder(tokenized.input_ids.to("cuda"))[0].squeeze()

# Generamos de nuevo el ruido
generator = torch.Generator().manual_seed(42)
noise = torch.randn((1, 4, 64, 64), generator=generator, dtype=torch.float16)

walk_steps = 30
batch_size = 4
batches = walk_steps // batch_size

walk_noise_x = torch.randn((4, 64, 64), generator=generator)
walk_noise_y = torch.randn((4, 64, 64), generator=generator)

walk_scale_x = torch.cos(torch.linspace(0, 2, walk_steps) * math.pi)
walk_scale_y = torch.sin(torch.linspace(0, 2, walk_steps) * math.pi)

noise_x = torch.tensordot(walk_scale_x, walk_noise_x, dims=0)
noise_y = torch.tensordot(walk_scale_y, walk_noise_y, dims=0)

noise = (noise_x + noise_y)
batched_noise = noise.to("cuda").to(torch.float16).chunk(batches)

In [ ]:
images = []
for noise_batch in batched_noise:
    prompt_batch = torch.stack([encoding] * noise_batch.shape[0])
    images += pipe(prompt_embeds=prompt_batch,
                   latents=noise_batch,
                   num_inference_steps=30).images

In [ ]:
export_as_gif("cows.gif", images)

In [ ]:
IImage("cows.gif")